Face Recognition

In [1]:
import face_recognition   
import cv2     
import os       
from matplotlib import pyplot as plt
import imutils
import numpy as np
from IPython.display import display, Javascript
from base64 import b64decode
from PIL import Image
from pathlib import Path
import cv2

In [2]:
def read_img(path):
   img = cv2.imread(path) 
   (h,w) = img.shape[:2]  
   width = 500            
   ratio = width / float(w) 
   height = int(h * ratio)  
   return cv2.resize(img,(width,height)) 

In [3]:
def known_names_list(known_names):
  present = []
  known_encodings = []
  knownn_dir = '/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known' 
  for file in os.listdir(knownn_dir):
    img = read_img(knownn_dir + '/' + file) 
    img_enc = face_recognition.face_encodings(img)[0] 
    known_encodings.append(img_enc) 
    known_names.append(file.split('.')[0])
    present.append(False)
  return known_encodings

In [4]:
import os

def list_names_in_class(known_names, known_encodings):
    unknown_dir = '/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/unknown' 
    list_names = []
    for file in os.listdir(unknown_dir):
        print("Processing", file)
        img_path = os.path.join(unknown_dir, file)
        img = read_img(img_path)
        img_encodings = face_recognition.face_encodings(img)
        if len(img_encodings) > 0:
            img_enc = img_encodings[0]
            results = face_recognition.compare_faces(known_encodings, img_enc)
            print(face_recognition.face_distance(known_encodings, img_enc))
            res = [i for i, val in enumerate(results) if val]
            name = known_names[res[0]]
            list_names.append(name)
            (top, right, bottom, left) = face_recognition.face_locations(img)[0]
            cv2.rectangle(img, (left, top), (right, bottom), (255, 255, 100), 2)
            cv2.putText(img, name, (left + 2, bottom + 20), cv2.FONT_HERSHEY_PLAIN, 3, (100, 0, 255), 2) 
            plt.imshow(img)
            plt.show()
        else:
            print('Não foi possível encontrar rostos na imagem', file)
            os.remove(img_path)  # Delete the image file

    return list_names


In [5]:

import csv
import datetime
a = []
known_names_list(a)

def export_csv(known_names, list_names):
    # obter a data atual
    date = datetime.datetime.now().strftime('%Y-%m-%d')

    # criar um arquivo CSV e escrever os cabeçalhos
    with open(date + '.csv', mode='w') as file:
        writer = csv.writer(file)
        writer.writerow(['Nome', date])

        # escrever as informações para cada pessoa
        for name in known_names:
            if name in list_names:
                present = True
            else:
                present = False
                list_names.append(name)
            writer.writerow([name, present])
            
        # escrever as informações para quem faltou
        for name in list_names:
            if name not in known_names:
                writer.writerow(['', name, False])




In [6]:
import os
import csv

def process_csv_files(folder_path):
    nome_vectors = {}

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, mode='r') as file:
                reader = csv.reader(file)
                headers = next(reader)  

                if len(headers) == 2 and headers[0] == "Nome" and headers[1].startswith("20"):
                    for row in reader:
                        nome = row[0]
                        value = row[1]

                        if nome not in nome_vectors:
                            nome_vectors[nome] = []

                        nome_vectors[nome].append(value)

    for nome, vector in nome_vectors.items():
        total_true = vector.count("True")
        total_false = vector.count("False")

        print(f"{nome}:")
        print(f"faltas: {total_false},", f"{(total_true/(total_false + total_true))*100}% de presença")


In [ ]:
import http.server
import socketserver
from PIL import Image
import io
import urllib.request

PORT = 8000
label = 1

import os

def known_names_list(known_names):
  present = []
  known_encodings = []
  knownn_dir = '/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known' 
  for file in os.listdir(knownn_dir):
    img = read_img(knownn_dir + '/' + file) 
    img_enc = face_recognition.face_encodings(img)[0] 
    known_encodings.append(img_enc) 
    known_names.append(file.split('.')[0])
    present.append(False)
  return known_encodings

import os

def list_names_in_class(known_names, known_encodings):
    unknown_dir = '/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/unknown' 
    list_names = []
    for file in os.listdir(unknown_dir):
        print("Processing", file)
        img_path = os.path.join(unknown_dir, file)
        img = read_img(img_path)
        img_encodings = face_recognition.face_encodings(img)
        if len(img_encodings) > 0:
            img_enc = img_encodings[0]
            results = face_recognition.compare_faces(known_encodings, img_enc)
            print(face_recognition.face_distance(known_encodings, img_enc))
            res = [i for i, val in enumerate(results) if val]
            if res:  # Check if res is not empty
                name = known_names[res[0]]
                list_names.append(name)
                (top, right, bottom, left) = face_recognition.face_locations(img)[0]
                cv2.rectangle(img, (left, top), (right, bottom), (255, 255, 100), 2)
                cv2.putText(img, name, (left + 2, bottom + 20), cv2.FONT_HERSHEY_PLAIN, 3, (100, 0, 255), 2) 
                plt.imshow(img)
                plt.show()
            else:
                print('Rosto desconhecido na imagem', file)
                os.remove(img_path)  # Delete the image file
        else:
            print('Não foi possível encontrar rostos na imagem', file)
            os.remove(img_path)  # Delete the image file

    return list_names


class MyHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/hello':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            with open('index.html', 'rb') as file:
                self.wfile.write(file.read())
        else:
            self.send_error(404)

    def do_POST(self):
        global label
        content_length = int(self.headers['Content-Length'])
        body = self.rfile.read(content_length)
        print("foto recebida")
        image = Image.open(io.BytesIO(body)).convert("RGB")
        image.show()

        image.save("unknown/"+str(label)+".jpg")
        label += 1
        
        known_names = []
        known_encodings = []
        known_encodings = known_names_list(known_names)

        list_names = []
        list_names = list_names_in_class(known_names, known_encodings)

        print(list_names)
        print(known_names)

        export_csv(known_names, list_names)
        folder_path = "/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server" 
        process_csv_files(folder_path)

        self.send_response(200)
        self.send_header('Content-type', 'image/jpeg')
        self.end_headers()
        #with open("unknown/"+str(label-1)+".jpg", 'rb') as file:
            #self.wfile.write(file.read())

with socketserver.TCPServer(("", PORT), MyHandler) as httpd:
    print("Serving at port", PORT)
    httpd.serve_forever()


In [263]:
import http.server
import socketserver
from PIL import Image
import socketserver
import io
import re
import urllib.request
import zipfile

PORT = 8000
label = 1

import os

def extract_filename_from_header(content_disposition):
    filename_match = re.search(r'filename="(.+)"', content_disposition)
    if filename_match:
        return filename_match.group(1)
    return None


def known_names_list(known_names):
    present = []
    known_encodings = []
    knownn_dir = '/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known' 
    for file in os.listdir(knownn_dir):
        img = read_img(knownn_dir + '/' + file) 
        img_enc = face_recognition.face_encodings(img)[0] 
        known_encodings.append(img_enc) 
        known_names.append(file.split('.')[0])
        present.append(False)
    return known_encodings

import os

def list_names_in_class(known_names, known_encodings):
    unknown_dir = '/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/unknown' 
    list_names = []
    for file in os.listdir(unknown_dir):
        print("Processing", file)
        img_path = os.path.join(unknown_dir, file)
        img = read_img(img_path)
        img_encodings = face_recognition.face_encodings(img)
        if len(img_encodings) > 0:
            img_enc = img_encodings[0]
            results = face_recognition.compare_faces(known_encodings, img_enc)
            print(face_recognition.face_distance(known_encodings, img_enc))
            res = [i for i, val in enumerate(results) if val]
            if res:  # Check if res is not empty
                name = known_names[res[0]]
                list_names.append(name)
                (top, right, bottom, left) = face_recognition.face_locations(img)[0]
                cv2.rectangle(img, (left, top), (right, bottom), (255, 255, 100), 2)
                cv2.putText(img, name, (left + 2, bottom + 20), cv2.FONT_HERSHEY_PLAIN, 3, (100, 0, 255), 2) 
                plt.imshow(img)
                plt.show()
            else:
                print('Rosto desconhecido na imagem', file)
                os.remove(img_path)  # Delete the image file
        else:
            print('Não foi possível encontrar rostos na imagem', file)
            os.remove(img_path)  # Delete the image file

    return list_names


class MyHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/hello':
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            with open('index.html', 'rb') as file:
                self.wfile.write(file.read())
        elif self.path == '/export-csv':
            folder_path = "/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server"

            # Cria um arquivo zip
            zip_file_path = os.path.join(folder_path, 'csv_files.zip')
            with zipfile.ZipFile(zip_file_path, 'w') as zip_file:
                for filename in os.listdir(folder_path):
                    if filename.endswith('.csv'):
                        file_path = os.path.join(folder_path, filename)
                        zip_file.write(file_path, os.path.basename(file_path))

            # Envia o arquivo zip como resposta
            with open(zip_file_path, 'rb') as file:
                content = file.read()

            self.send_response(200)
            self.send_header('Content-type', 'application/zip')
            self.send_header('Content-Disposition', 'attachment; filename="csv_files.zip"')
            self.send_header('Content-Length', str(len(content)))
            self.end_headers()
            self.wfile.write(content)

        else:
            self.send_error(404)

    def do_POST(self):
        global label
        if self.path == '/save-image':
            content_length = int(self.headers['Content-Length'])
            body = self.rfile.read(content_length)

            print("Foto recebida")

            content_disposition = self.headers.get('Content-Disposition')

            if content_disposition:
                filename = extract_filename_from_header(content_disposition)
                if filename:
                    file_path = os.path.join("/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known", filename)
                else:
                    file_path = "/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known/teste"
            else:
                file_path = "/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known/teste"

            with open(file_path, 'wb') as file:  # Use 'wb' para abrir o arquivo em modo binário
                file.write(body)

            self.send_response(200)
            self.send_header('Content-type', 'image/jpeg')
            self.end_headers()

            label += 1

        else:
            content_length = int(self.headers['Content-Length'])
            body = self.rfile.read(content_length)
            print("foto recebida")
            image = Image.open(io.BytesIO(body)).convert("RGB")
            image.show()

            image.save("unknown/"+str(label)+".jpg")
            label += 1
            
            known_names = []
            known_encodings = []
            known_encodings = known_names_list(known_names)

            list_names = []
            list_names = list_names_in_class(known_names, known_encodings)

            print(list_names)
            print(known_names)

            export_csv(known_names, list_names)
            folder_path = "/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server" 
            process_csv_files(folder_path)

            self.send_response(200)
            self.send_header('Content-type', 'image/jpeg')
            self.end_headers()

Handler = MyHandler

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print("Serving at port", PORT)
    httpd.serve_forever()


Serving at port 8000


127.0.0.1 - - [05/Jul/2023 14:15:23] "GET /hello HTTP/1.1" 200 -


Tamanho do corpo recebido: 5005
Foto recebida
attachment; filename="cabelinho.jpg"
/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known/cabelinho.jpg
aaaaaaaaaaaaaaaaaa


127.0.0.1 - - [05/Jul/2023 14:15:26] "POST /save-image HTTP/1.1" 200 -


Tamanho do corpo recebido: 6355
Foto recebida
attachment; filename="lula.jpg"
/home/nicholas/Área de Trabalho/TCC/nicholas (1)/server/known/lula.jpg
aaaaaaaaaaaaaaaaaa


127.0.0.1 - - [05/Jul/2023 14:15:29] "POST /save-image HTTP/1.1" 200 -


KeyboardInterrupt: 